In [2]:
from src.data.dataset import prepare_dataset
from src.data.loaders import load_prices
from src.training.evaluate import run_baselines

# 1) Build dataset
D = prepare_dataset()

# 2) Optional: load prices aligned to index
px = load_prices(D["meta"]["ticker"], D["meta"].get("interval", "1d"))

# 3) Run baselines
df = run_baselines(D, prices=px, use_xgb=False, pooling_lr="last")

# 4) Display
df

# 5) Save
import pandas as pd
out = "data/artifacts/baselines_metrics.csv"
pd.DataFrame(df).to_csv(out, index=False)
print("Saved:", out)

TypeError: prepare_dataset() missing 1 required positional argument: 'cfg'